1. https://realpython.com/k-means-clustering-python/
2. https://medium.com/@rohithramesh1991/unsupervised-text-clustering-using-natural-language-processing-nlp-1a8bc18b048d

In [ ]:
import pandas as pd
from nltk import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pickle as pkl
import matplotlib.pyplot as plt

# If needed run and download NLTK Lib to run the code
# import nltk
# nltk.download()


In [101]:
df = pd.read_csv('../../../DataSets/Hotel_Reviews.csv')

In [102]:
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [113]:
df_review_n = pd.DataFrame()
df_review_n["review"] = df["Negative_Review"]
df_review_n["output"] = 0


In [114]:
df_review_n.head()

,review,output
0,I am so angry that i made this post available...,0
1,No Negative,0
2,Rooms are nice but for elderly a bit difficul...,0
3,My room was dirty and I was afraid to walk ba...,0
4,You When I booked with your company on line y...,0


In [115]:
df_review_p = pd.DataFrame()
df_review_p["review"] = df["Positive_Review"]
df_review_p["output"] = 1


In [116]:
df_review_p.head()

,review,output
0,Only the park outside of the hotel was beauti...,1
1,No real complaints the hotel was great great ...,1
2,Location was good and staff were ok It is cut...,1
3,Great location in nice surroundings the bar a...,1
4,Amazing location and building Romantic setting,1


In [117]:
final_df = pd.concat([df_review_p, df_review_n], ignore_index=True)

In [118]:
final_df = final_df.sample(frac=1).reset_index(drop=True)

In [119]:
final_df.head()

,review,output
0,Excellent room overlooking river vet comforta...,1
1,The advertised spa area didn t have a spa Onl...,0
2,everything was great,1
3,Location,1
4,The bellmen doormen and the reception staff a...,1


In [120]:
final_df.dropna(inplace=True)

In [122]:
final_df["review_token"] = [word_tokenize(post) for post in final_df['review']]


In [123]:
post_new = []
for each_row in final_df["review_token"]:
    post_new.append( [v for v in each_row if v.isalpha()])

In [124]:
post_new_alpha = []

stop_words = set(stopwords.words('english'))

for each_row in post_new:
    post_new_alpha.append( [sw for sw in each_row if sw not in stop_words ])

In [127]:
final_df

,review,output,review_token,review_token_cleaned
0,Excellent room overlooking river vet comforta...,1,"[Excellent, room, overlooking, river, vet, com...","[Excellent, room, overlooking, river, vet, com..."
1,The advertised spa area didn t have a spa Onl...,0,"[The, advertised, spa, area, didn, t, have, a,...","[The, advertised, spa, area, spa, Only, sauna]"
2,everything was great,1,"[everything, was, great]","[everything, great]"
3,Location,1,[Location],[Location]
4,The bellmen doormen and the reception staff a...,1,"[The, bellmen, doormen, and, the, reception, s...","[The, bellman, doorman, reception, staff, extr..."
...,...,...,...,...
1031471,I have read it in previous reviews already bu...,0,"[I, have, read, it, in, previous, reviews, alr...","[I, read, previous, review, already, little, s..."
1031472,The view was amazing and the location was per...,1,"[The, view, was, amazing, and, the, location, ...","[The, view, amazing, location, perfect, Close,..."
1031473,Everything of above,1,"[Everything, of, above]",[Everything]
1031474,We loved everything about this hotel The styl...,1,"[We, loved, everything, about, this, hotel, Th...","[We, loved, everything, hotel, The, stylish, i..."


In [129]:
desc_new_lem = []
lem = WordNetLemmatizer()
for each_row in post_new_alpha:
    desc_new_lem.append([lem.lemmatize(word) for word in each_row])

final_df["review_token_cleaned"] = desc_new_lem
final_df["review_token_cleaned"] = [
    " ".join(desc) for desc in final_df['review_token'].values]


In [130]:
final_df.drop(["review", "review_token"], axis=1, inplace=True)


In [131]:
final_df.head()

,output,review_token_cleaned
0,1,Excellent room overlooking river vet comfortab...
1,0,The advertised spa area didn t have a spa Only...
2,1,everything was great
3,1,Location
4,1,The bellmen doormen and the reception staff al...


In [132]:
tfi = TfidfVectorizer(max_features=1000)
x = tfi.fit_transform(final_df["review_token_cleaned"]).toarray()
X = pd.DataFrame(x)

In [134]:
y = np.array(final_df['output'])

In [135]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)


In [136]:
model = KMeans(init='random', n_clusters=2, n_init=100, max_iter=300, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [138]:
sum_of_squared_distances = []
K = range(1, 150)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(X_train)
    sum_of_squared_distances.append(km.inertia_)


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


KeyboardInterrupt: 